### 미처 처리하지 못한 한자, 특수문자 제거

In [7]:
import re
import hanja
import pandas as pd

path = '../../v1.3.1_aug3/train.csv'
train = pd.read_csv(path)
for _, data in train.iterrows():
    text = data['text']
    text = hanja.translate(text, 'substitution')
    text = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", text)
    train.loc[train['ID']==data['ID'], 'text'] = text
train.to_csv(path, index=False)

### v1.1.2 데이터 확인
- LLaMA, 정규표현식으로 1차 처리한 데이터들

In [8]:
import numpy as np
import pandas as pd

score = pd.DataFrame(columns=['target', 'total', 'train', 'valid', 'f1'])
score['target'] = np.arange(7)

v112_total = pd.read_csv('../../v1.1.2/train.csv')
v112_val_pred = pd.read_csv('../../v1.1.2/valid_output.csv')

val_ids = list(v112_val_pred['ID'].values)
v112_train = v112_total[~v112_total['ID'].str.contains('|'.join(val_ids))]
v112_val_answer = v112_total[v112_total['ID'].str.contains('|'.join(val_ids))]

label = range(7)
for l in label:
    score.loc[score['target']==l, 'total'] = len(v112_total[v112_total['target']==l])
    score.loc[score['target']==l, 'train'] = len(v112_train[v112_train['target']==l])
    score.loc[score['target']==l, 'valid'] = len(v112_val_pred[v112_val_pred['target']==l])

    val_correct = v112_val_pred[(v112_val_pred['target']==l) & v112_val_pred['ID'].str.contains('|'.join((v112_val_answer[v112_val_answer['target']==l]['ID'].values)))]
    recall = len(val_correct) / (len(v112_val_answer[v112_val_answer['target']==l]))
    precision = len(val_correct) / (len(v112_val_pred[v112_val_pred['target']==l]))

    score.loc[score['target']==l, 'f1'] = 2 * precision * recall / (precision + recall)

score

FileNotFoundError: [Errno 2] No such file or directory: '../../v1.1.2/valid_output.csv'

텍스트 정제

In [ ]:
text_polluted = v112_total[(0.3 <= v112_total['polluted_lv']) & (v112_total['polluted_lv'] < 0.35)]
# print(len(text_polluted))
# text_polluted.sample(10)
clean_text = v112_total[v112_total['polluted_lv'] < 0.3]
# clean_text.to_csv('../../v1.3.0/train.csv', index=False)
print(len(clean_text))
clean_text.sample(5)

1835


,ID,text,target,polluted_lv
2111,ynat-v1_train_01444,SKT 갤럭시 A8 스타 단독 출시,3,0.263158
976,ynat-v1_train_02049,전국 아침 기온 최대 7도 뚝…주말까지 쌀쌀,0,0.043478
1475,ynat-v1_train_00241,?0 V3 모바일 시큐리티 :50k T]로드 돌파,4,0.291667
1664,ynat-v1_train_00632,北|마라톤 선수단 올림hz참가 자격위Om?B행위,2,0.208333
129,ynat-v1_train_00283,부산국제코미디페스티벌에 참여합니다,0,0.000000


### 각 버전의 데이터셋으로 학습한 모델로 CleanLab

In [47]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
tokenizers = ['../../model-v1.1.2', '../../model-v1.2.0', '../../model-v1.2.2']
models = ['../../model-v1.1.2/checkpoint-140', '../../model-v1.2.0/checkpoint-146', '../../model-v1.2.2/checkpoint-164']

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm

# clean_text = pd.read_csv('../../../data/train.csv')
# clean_text = pd.read_csv('../../v1.3.0/train_1835.csv')

total_preds = []
for t, m in zip(tokenizers, models):
    tokenizer = AutoTokenizer.from_pretrained(t)
    model = AutoModelForSequenceClassification.from_pretrained(m, num_labels=7).to(DEVICE)
    model.eval()

    preds = []
    for _, data in tqdm(clean_text.iterrows(), total=len(clean_text)):
        tokenized = tokenizer(data['text'], padding='max_length', max_length=50, truncation=True, return_tensors='pt').to(DEVICE)
        with torch.no_grad():
            logits = model(**tokenized).logits.cpu().numpy()
            softmax = [np.exp(x)/np.sum(np.exp(logits)) for x in logits]
            preds.append(np.array(softmax[0]))
    preds = np.array(preds)
    print(preds.shape)

    total_preds.append({"data_ver": t.split('../../')[1], "preds": preds})

  0%|          | 0/1835 [00:00<?, ?it/s]

(1835, 7)


  0%|          | 0/1835 [00:00<?, ?it/s]

(1835, 7)


  0%|          | 0/1835 [00:00<?, ?it/s]

(1835, 7)


In [52]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from cleanlab.filter import find_label_issues

total_issues = []
for preds in total_preds:
    pred_probs = preds['preds']
    ordered_label_issues = find_label_issues(
        labels=clean_text['target'],
        pred_probs=pred_probs,
        return_indices_ranked_by='self_confidence'
    )
    total_issues.append({"data_ver": preds["data_ver"], "issues": ordered_label_issues})

    print(f"total {len(ordered_label_issues)}")

total 435
total 470
total 518


In [64]:
import copy

for issues in total_issues:
    data = copy.deepcopy(clean_text)
    for issue in issues['issues']:
        print('input text:', clean_text.iloc[issue]['text'])
        print('label:', clean_text.iloc[issue]['target'])
        pred = np.argmax(pred_probs[issue])
        print('pred:', pred)
        data.loc[clean_text['text']==clean_text.iloc[issue]['text'], 'target'] = pred

        print('----------------------------------------')
    print("!+!+!+!+!+!+!+!+!+!+!+!+!+!+!+!+!+!+!+!+!+!+!+!+!+!+!+!+")
    data_ver = issues['data_ver'].split('-')[1]
    data.to_csv(f'../../v1.3.0/train_{data_ver}.csv', index=False)

input text: 호날두 vs 메시 266번째 엘클라시코가 온다
label: 5
pred: 1
----------------------------------------
input text: 메시·호날두 UEFA 올해의 팀에 선정…EPL 선수 제로
label: 5
pred: 1
----------------------------------------
input text: 칠레 청년 사자 우리서 자살 기도…구조과정서 사자 2마리 사살
label: 5
pred: 6
----------------------------------------
input text: 기성용정우영 없는 중원 구자철황인범 라인 뜰까
label: 3
pred: 1
----------------------------------------
input text: 탱크·전투 헬기가 바닷속에…요르단 수중 군사 박물관 공개
label: 5
pred: 6
----------------------------------------
input text: 홈런 2방에 벌랜더 와르르…MLB 양키스 ALCS서 기사회생
label: 0
pred: 1
----------------------------------------
input text: UEFA 파리 생제르맹 FFP 위반 여부 조사 착수
label: 0
pred: 1
----------------------------------------
input text: MLB.com 다저스 3년 연속 WS 유력하지만 우승은 글쎄…
label: 4
pred: 1
----------------------------------------
input text: UEFA 챔스리그 4강 마드리드 더비 성사…레알 vs 아...
label: 0
pred: 1
----------------------------------------
input text: 우리은행 박혜진 연봉 3억원으로 2년 연속 연봉 퀸
label: 5
pred: 1
----------------------------

In [85]:
label_cleaned = pd.read_csv('../../v1.3.0/train.csv')
label_cleaned['target'].value_counts(sort=True)

aug_3 = pd.read_csv('../../v1.2.0_aug3/train.csv')
aug_3 = aug_3[aug_3['ID'].str.contains('aug')]

label_cleaned_aug_3 = pd.concat([label_cleaned, aug_3])
# label_cleaned_aug_3.to_csv('../../v1.3.1_aug3/train.csv', index=False)
label_cleaned_aug_3['target'].value_counts()

target
1    315
6    308
0    286
5    279
4    272
2    259
3    216
Name: count, dtype: int64